Cloning delta tables

In [0]:
%sql
use catalog `get_started`;
use schema `labuser`;

SHOW TABLES

In [0]:
%sql
CREATE OR REPLACE TABLE historical_sales_deep_clone
DEEP CLONE historical_sales_bronze;

In [0]:
%sql
CREATE OR REPLACE TABLE historical_sales_shallow_clone
SHALLOW CLONE historical_sales_bronze;

Complete Overwrites - are atomically replacement of data in a table. There ar benefits of overwriting tables instead of deleting and recreating tables

In [0]:
%sql
DESCRIBE HISTORY current_employees_bronze;

In [0]:
%sql
INSERT OVERWRITE current_employees_bronze
SELECT * FROM read_files(
    '/Volumes/get_started/labuser/myfiles/employees.csv',
    format => 'csv',
    header => true,
    inferSchema => true
);

DESCRIBE HISTORY current_employees_bronze;

### MERGE

```sql 
MERGE INTO target a
USING source b
ON {merge_condition}
WHEN MATCHED THEN {matched_action}
WHEN NOT MATCHED THEN {not_matched_action}
```

In [0]:
%sql
MERGE INTO users_table as a 
USING users_update_table as b
ON a.user_id = b.user_id
WHEN MATCHED AND a.email IS NULL AND b.email IS NOT NULL THEN
 UPDATE SET email = b.email, updated_time = b.updated_time
WHEN NOT MATCHED THEN 
  INSERT (user_id,email,updated_time) 
  VALUES (b.user_id, b.email, b.updated_time)

### Insert-Only Merge for deduplication

In [0]:
%sql
MERGE INTO events_table as a 
USING events_update_table as b
ON a.user_id = b.user_id AND a.event_time = b.event_time -- checking to make sure we don't have duplicate records
WHEN NOT MATCHED THEN
 INSERT *

Filtering and Renaming Columns, eg (order_id as id)

In [0]:
%sql
CREATE OR REPLACE TABLE purchases AS
SELECT order_id as id, transaction_date as date, purchase_revenue as revenue
FROM purchases_bronze;

SELECT * FROM purchases LIMIT 5;

DECLARE SCHEMA WITH GENERATED COLUMNS

In [0]:
%sql
CREATE OR REPLACE TABLE purchase_dates (
  id STRING,
  transaction_timestamp STRING,
  price STRING,
  date DATE GENERATED ALWAYS AS(
    cast(cast(transaction_timestamp as timestamp) as date))
    COMMENT "generated based on transaction_timestamp column"
  

DATA CONSTRAINTS (eg; NOT NULL, NULL)

In [0]:
%sql
ALTER TABLE purchase_dates ADD CONSTRAINT valid_date CHECK (date > '2020-01-01')

In [0]:
## do not forget to cleanup after